## The Zoning Problem 
The aim of this notebook is to generate the optimal zoning by formulating a linear assignment problem and using google
OR-Toolsfor solving the optimization problem.
The agent behaviours are taken from the previous study. The occupation and env lattices are random for the test problem .

### The Test Problem 
There are five agents :[Blue,Green,Yellow,Red,Violet]
The number of voxels which each agent occupies are [B1,G1,Y1,R1,V1]
Total number of voxels in a lattice = x 
Value lattice for the agents are =[Bv],[Gv],[Yv],[Rv],[Vv]

####  Aim 
: To find the best combination of Zones to achieve the maximum occupancy value
#### Steps

1. Generate the lattices , Agent behaviours, agents, 
2. Find the possible origin locations for the agents
3. Simulate occupancy behaviour and retrieve the cost of occupancy for each agent at each position
4. Generate the Cost matrix
5. Use the MIP Solver to optimise the combination to get the permutation matrix 


## Initilization

In [1]:
import os
import itertools
import sys
from math import factorial as fac
sys.path.append("D:/TU_Delft/Msc_Building_Technology/Semester_3/Graduation/Aditya_Graduation_Project_BT/06_Libraries")
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
np.random.seed(0)
np.set_printoptions(threshold=sys.maxsize)
import networkx as nx
from itertools import combinations
import pickle

## Base Lattices

In [2]:
# loading the lattice from csv
lattice_path = os.path.relpath('voxelized_envelope_6m_voxel_size.csv')
avail_lattice_base = tg.lattice_from_csv(lattice_path)
avail_lattice = avail_lattice_base*1
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)

In [3]:
flattened_lattice = avail_lattice_base.flatten()

In [7]:
len(np.argwhere(flattened_lattice==1))

1359

## Env Lattices

In [4]:
Private_sector_Owned = (pickle.load( open( "Privately Owned Housing.p", "rb" ) ))
Social_Sector_Rental = (pickle.load( open( "Social Sector Rental Housing.p", "rb" ) ))
Free_Sector_rental = (pickle.load( open( "Free Sector Rental Housing.p", "rb" ) )) 
Restaurants_and_Cafes = (pickle.load( open( "Restaurants and Cafe.p", "rb" ) )) 
Offices = (pickle.load( open( "Offices.p", "rb" ) )) 
Retail = (pickle.load( open( "Retail Stores.p", "rb" ) )) 
Parking = (pickle.load( open( "Parking.p", "rb" ) )) 

FileNotFoundError: [Errno 2] No such file or directory: 'Privately Owned Housing.p'

## Stencils

In [ ]:
# creating neighborhood definition
stencil_von_neumann = tg.create_stencil("von_neumann", 1, 1)
stencil_von_neumann.set_index([0,0,0], 0)
#print(stencil_von_neumann)

In [ ]:
# creating neighborhood definition 
stencil_squareness = tg.create_stencil("moore", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness[0,:,:] = 0 
stencil_squareness[2,:,:] = 0
stencil_squareness.set_index([0,0,0], 0)
stencil_squareness_t = np.transpose(stencil_squareness) 
#print(stencil_squareness_t)

In [ ]:
# creating neighborhood definition 
stencil_squareness_von = tg.create_stencil("von_neumann", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness_von[0,:,:] = 0 
stencil_squareness_von[2,:,:] = 0
stencil_squareness_von.set_index([0,0,0], 0)
stencil_squareness_von_t = np.transpose(stencil_squareness_von) 
#print(stencil_squareness_von)

In [ ]:
stencil_cuboid = tg.create_stencil("moore", 1, 1)
stencil_cuboid.set_index([0,0,0], 0)
#print(stencil_cuboid)

In [ ]:
Stencil_Find_neighbours_cuboid = tg.create_stencil("moore", 1, 1)

## Deriving all possible agent Center points

In [ ]:
available_index_list= list(np.argwhere(flattened_lattice==1).flatten())

In [ ]:
Selected_positions=[]
for item in range(19,1359,19):
    Selected_positions.append(available_index_list[item]) 

In [ ]:
selected_neighbours= avail_lattice_base.find_neighbours(Stencil_Find_neighbours_cuboid)

In [ ]:
#np.array(np.unravel_index(Selected_positions,init_avail_lattice.shape)).T

In [ ]:
def all_possible_origins(flat_lattice,number_of_points_needed):
    reshaped_lattice= flat_lattice.reshape(99,4,16)*1
    reshaped_lattice_aggregated_value =np.sum(reshaped_lattice,axis=2)
    array = reshaped_lattice_aggregated_value.flatten()
    temp = array.argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(array))
    
    indices =list(range(6,flat_lattice.size,16))
    points_1D=[]
    for points in range(number_of_points_needed):
        index= np.where(ranks==points)
        points_1D.append(indices[int(index[0])] )  
    return points_1D


In [ ]:
## The number of voxels which each agent has to occupy There are five agents :[Blue,Green,Yellow,Red,Violet]
## The number of voxels which each agent occupies are [B1,G1,Y1,R1,V1] 
mass_size = np.count_nonzero((avail_lattice==1)) 
P_H = 415 # Private Housing
S_H= 301 # Social Housing
F_H=403 # Free sector Housing
R_C= 19 # Restaurant and Cafe 
R= 73 # Retail stores
O=93 # Offices
P=51 # Parking


In [ ]:
#Divide the available lattice into grids to generate the necessary locations for origins??

In [ ]:
def ranking_origins(point_id,value_lattice,origin_number):
    selected_neighbours= avail_lattice_base.find_neighbours(Stencil_Find_neighbours_cuboid)[point_id]
    neighbour_vals_for_agent= value_lattice.flatten()[selected_neighbours]
    neighbour_vals_aggregated_for_agent =np.sum(neighbour_vals_for_agent,axis=1)
    all_points = np.array(point_id)
    
    array = neighbour_vals_aggregated_for_agent
    temp = array.argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(array))
    
    a_lattice = ranks.flatten()
    Co_ordinates = []
    for item in range(0,origin_number):
        Co_ordinates.append(all_points[np.argwhere(a_lattice==item)].flatten()[0])    
        
    return Co_ordinates

In [ ]:
ranking_origins(Selected_positions,Private_sector_Owned,10)

In [ ]:
Agent_1_origins = ranking_origins(Selected_positions,Private_sector_Owned,21)
Agent_2_origins = ranking_origins(Selected_positions,Social_Sector_Rental,21)
Agent_3_origins = ranking_origins(Selected_positions,Free_Sector_rental,21)
Agent_4_origins = ranking_origins(Selected_positions,Restaurants_and_Cafes,21)
Agent_5_origins = ranking_origins(Selected_positions,Offices,21)
Agent_6_origins = ranking_origins(Selected_positions,Retail,21)
Agent_7_origins = ranking_origins(Selected_positions,Parking,21)

In [ ]:
All_agent_origins = np.concatenate((Agent_1_origins,Agent_2_origins,Agent_3_origins,Agent_4_origins,Agent_5_origins,Agent_6_origins,Agent_7_origins))

In [ ]:
all_permutations= All_agent_origins.reshape(7, len(Agent_1_origins)).T

In [ ]:
# agent class
class agent():
    def __init__(self, origin, stencil, id):

        # define the origin attribute of the agent and making sure that it is an intiger
        self.origin = np.array(origin).astype(int)
        # define old origin attribute and assigning the origin to it as the initial state
        self.old_origin = self.origin
        # define stencil of the agent
        self.stencil = stencil
        #define agent id
        self.id = id

    # definition of random/argmax occupancy on a 2d squarish stencil 
    def random_occupy_squareness(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
     
      # definition of random/argmax occupancy on a 3d cubish stencil
    def random_occupy_cubish(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
      
        
    def random_occupy_cubish_von_neumann(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_cuboid, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
        
    def argmax_occupy_von_neumann(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)

        
     # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_moore(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        #print(neighs)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)

        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]               
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        #print(neighs_availibility)
        
        # find env values of all neighbours
        all_neighs_value = env.value.flatten()[neighs]
        all_neighs_value_mod = np.copy(all_neighs_value)
        
        
        #finding number of neighbours and bumping the values based on adjacency for a 9 neighbourhood
        
        #print(neighbourhood_details)
        one = neighs_availibility[1] + neighs_availibility[2] 
        two = neighs_availibility[0] + neighs_availibility[2] 
        three = neighs_availibility[1] + neighs_availibility[4] 
        four = neighs_availibility[0] + neighs_availibility[6] 
        five = neighs_availibility[2] + neighs_availibility[7] 
        six = neighs_availibility[3] + neighs_availibility[6] 
        seven = neighs_availibility[5] + neighs_availibility[7] 
        eight = neighs_availibility[6] + neighs_availibility[4] 
        neighbourhood_details = [one,two,three,four,five,six,seven,eight]
        
        #print(neighbourhood_details)
        for detail in range(len(neighs_availibility)-1):
            neighbourhood_condition = neighbourhood_details[detail] 
            #print(neighbourhood_condition)
            if neighbourhood_condition == 3:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + one_neighbour_factor
            elif neighbourhood_condition == 4:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + two_neighbour_factor
            else:
                all_neighs_value_mod[detail] = all_neighs_value_mod[detail]
        #print(all_neighs_value_mod)   
        

        neighs_value_flattened = env.value.flatten()
        for val_mod in all_neighs_value_mod:
            for neigh in neighs :
                neighs_value_flattened[neigh]=val_mod
        
        
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = neighs_value_flattened[free_neighs]
        
        #print(free_neighs_value)
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    
    
         # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_von_neumann(self, env):
        
         # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    def one_neighbour_occupy_squareness_behaviour (self,env):        
        value_lattice_flat = env.value.flatten()
        sqr_factor = 10.1
        sqr_shift = 10.0
        init_loc = self.origin
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        
        agn_locs = [list(init_loc)]
        all_neighs =[] 
        avail_lattice_flat = env.availibility.flatten()
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        all_neighs.append(neighs)
        env.neigh_squareness.append(neighs)

        neighs_flatten = np.array(env.neigh_squareness).flatten()
        #print(neighs_flatten)
        neighs_availability = avail_lattice_flat[neighs_flatten]
  
        # keep the available ones only
        avail_neighs = neighs_flatten[neighs_availability==1] 
        
        if len(avail_neighs)== 0 :
            avail_neighs = free_neighs_full_lattice
        else: 
            avail_neighs= avail_neighs
        #print(avail_neighs)
        avail_unq_neighs, avail_unq_neighs_count = np.unique(avail_neighs, return_counts=True)
        #print(avail_unq_neighs)
        #print(avail_unq_neighs_count)
        neighs_unq_base_value = value_lattice_flat[avail_unq_neighs]
        neigh_sqr_evaluation = np.power(sqr_factor, (avail_unq_neighs_count - 1)) * neighs_unq_base_value + sqr_shift
        #neigh_sqr_evaluation = neighs_unq_base_value + sqr_shift * (avail_unq_neighs_count - 1)


        selected_neigh_index = np.argmax(neigh_sqr_evaluation)
        selected_neigh_1D_id = avail_unq_neighs[selected_neigh_index]
        #selected_neigh_3D_id = np.unravel_index(selected_neigh_1D_id,bounds.shape )

        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh_1D_id, env.availibility.shape)).flatten()
   
        
        
    def one_neighbour_occupy_cubish_behaviour (self,env):        
        value_lattice_flat = env.value.flatten()
        sqr_factor = 10.1
        sqr_shift = 10.0
        init_loc = self.origin
        neighs_availibility_full_lattice = env.availibility.flatten()
        free_neighs_full_lattice = np.argwhere(neighs_availibility_full_lattice==1).flatten()
        
        agn_locs = [list(init_loc)]
        all_neighs =[] 
        avail_lattice_flat = env.availibility.flatten()
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        all_neighs.append(neighs)
        env.neigh_cubish.append(neighs)

        neighs_flatten = np.array(env.neigh_cubish).flatten()
        #print(neighs_flatten)
        neighs_availability = avail_lattice_flat[neighs_flatten]
  
        # keep the available ones only
        avail_neighs = neighs_flatten[neighs_availability==1] 
        
        if len(avail_neighs)== 0 :
            avail_neighs = free_neighs_full_lattice
        else: 
            avail_neighs= avail_neighs
                
        #print(avail_neighs)
        avail_unq_neighs, avail_unq_neighs_count = np.unique(avail_neighs, return_counts=True)
        #print(avail_unq_neighs)
        #print(avail_unq_neighs_count)
        neighs_unq_base_value = value_lattice_flat[avail_unq_neighs]
        
        #neigh_sqr_evaluation = np.power(sqr_factor, (avail_unq_neighs_count - 1)) * neighs_unq_base_value + sqr_shift
        neigh_sqr_evaluation = neighs_unq_base_value + sqr_shift * (avail_unq_neighs_count - 1)

       # print(neighs_unq_base_value) 
        selected_neigh_index = np.argmax(neigh_sqr_evaluation)
        selected_neigh_1D_id = avail_unq_neighs[selected_neigh_index]
        #selected_neigh_3D_id = np.unravel_index(selected_neigh_1D_id,bounds.shape )       
            
        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh_1D_id, env.availibility.shape)).flatten()
        #print(selected_neigh_1D_id)
        #print(free_neighs_full_lattice)

            

In [ ]:
# Agent init class

def initialize_agents_random_origin (stencil,avail_lattice):
    #finding origin 
    agn_num = 1
    occ_lattice = avail_lattice*0 -1
    avail_flat = avail_lattice.flatten()
    avail_index = np.array(np.where(avail_lattice == 1)).T
    select_id = np.random.choice(len(avail_index), agn_num)
    agn_origins = tuple(avail_index[select_id].flatten()) 

    # Defining agents
    myagent = agent(agn_origins, stencil, select_id)

    return myagent

def initialize_agents_fixed_origin (stencil,avail_lattice,origin):
    #finding origin 
    agn_origins = np.unravel_index(origin,avail_lattice.shape)
    select_id = origin
    # Defining agents
    myagent = agent(agn_origins, stencil, select_id)

    return myagent

In [ ]:
# environment class
class environment():
    def __init__(self, lattices, agents,number_of_iterations,method_name):
        self.availibility = lattices["availibility"]
        self.value = lattices["enviornment"]
        self.agent_origin = self.availibility 
        self.agents = agents
        self.update_agents()
        self.number_of_iterations = number_of_iterations
        self.method_name = method_name
        self.neigh_cubish = []
        self.neigh_squareness = []
    def update_agents(self):
        # making previous position available
      #  self.availibility[tuple(self.agents.old_origin)] = self.availibility[tuple(self.agents.old_origin)] * 0 + 1
        # removing agent from previous position
        self.agent_origin[tuple(self.agents.old_origin)] *= 0+1
        # making the current position unavailable
        self.availibility[tuple(self.agents.origin)] = self.agents.id
        # adding agent to the new position 
        self.agent_origin[tuple(self.agents.origin)] = self.agents.id
    
    def random_occupy_squareness_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_squareness(self)
        # update the agent states in environment
        self.update_agents()
        
    def random_occupy_cubish_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish(self)
        # update the agent states in environment
        self.update_agents()
    
    def random_occupy_cubish_von_neumann_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
    
    def argmax_occupy_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.argmax_occupy_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_moore(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_moore(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
      
    def one_neighbour_occupy_cubish_behaviour(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_cubish_behaviour(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_behaviour(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_behaviour(self)
        # update the agent states in environment
        self.update_agents()
        

In [ ]:
def calculate_cost_for_positions(Location_list,Enviornment_lattice,number_of_cells_to_be_occupied):
    All_viz_options = []
    for indexing, item in enumerate(Location_list):
        #print(indexing)
        Agent = agent(np.unravel_index(item,init_avail_lattice.shape),stencil_cuboid,9)

        occ_lattice_sim = tg.to_lattice(np.copy(avail_lattice), avail_lattice)

        env = {"availibility": occ_lattice_sim,"enviornment": Enviornment_lattice}


        env_1 = environment(env,Agent,number_of_cells_to_be_occupied,"one_neighbour_occupy_cubish_behaviour")

        env_availability_viz = []
        env_list =[env_1]
        number_steps = max(map(lambda e:e.number_of_iterations,env_list))    
        for a in range(number_steps):
            # print(env.availibility)
            #print(env.agent_origin)

            for e in env_list:
                if a < e.number_of_iterations :
                    #print(a)
                    #print(e.number_of_iterations)
                    if e.method_name == "one_neighbour_occupy_squareness_moore":
                        e.one_neighbour_occupy_squareness_moore()

                    elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                        e.one_neighbour_occupy_cubish_agents()

                    elif e.method_name == "random_occupy_squareness_agents" :
                        e.random_occupy_squareness_agents()

                    elif e.method_name == "random_occupy_cubish_agents" :
                        e.random_occupy_cubish_agents()  

                    elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                        e.random_occupy_cubish_von_neumann_agents()                           

                    elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                        e.one_neighbour_occupy_squareness_von_neumann()                


                    elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                        e.one_neighbour_occupy_squareness_behaviour()  

                    elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                        e.one_neighbour_occupy_cubish_behaviour()  

                    elif e.method_name == "argmax_occupy_von_neumann" :
                        e.argmax_occupy_von_neumann()  

        env_availability_viz.append(e.availibility)

        All_viz_options.append(env_availability_viz)

    all_viz_array= np.array(All_viz_options)
    all_viz_array_shape = all_viz_array.shape
    reshaped_all_viz= all_viz_array.reshape(all_viz_array_shape[0]*all_viz_array_shape[1],all_viz_array_shape[2],all_viz_array_shape[3],all_viz_array_shape[4])
    sum_of_values = []
    for lattice in reshaped_all_viz:
        indexes= np.argwhere(lattice.flatten()==9).flatten()
        values= env["enviornment"].flatten()[indexes]
        total = int(np.sum(values))
        sum_of_values.append(total)
    return sum_of_values

In [ ]:
Cost_for_PH = np.array(calculate_cost_for_positions(Selected_positions,Private_sector_Owned,20)) # Private Housing
Cost_for_SH = np.array(calculate_cost_for_positions(Selected_positions,Social_Sector_Rental,20)) # Social Housing
Cost_for_FH = np.array(calculate_cost_for_positions(Selected_positions,Free_Sector_rental,20)) # Free sector Housing
Cost_for_RC = np.array(calculate_cost_for_positions(Selected_positions,Restaurants_and_Cafes,20)) # Restaurant and Cafe 
Cost_for_R = np.array(calculate_cost_for_positions(Selected_positions,Offices,20)) # Retail stores
Cost_for_O = np.array(calculate_cost_for_positions(Selected_positions,Retail,20)) # Offices
Cost_for_P = np.array(calculate_cost_for_positions(Selected_positions,Parking,20)) # Parking

In [ ]:
cost_matrix = np.concatenate ((np.tile(Cost_for_PH,21),np.tile(Cost_for_SH,16),np.tile(Cost_for_FH,21),np.tile(Cost_for_RC,1),np.tile(Cost_for_R,4),np.tile(Cost_for_O,5),np.tile(Cost_for_P,3)))

In [ ]:
cost_matrix_reshaped = cost_matrix.reshape (int(cost_matrix.size/71), 71 )

In [ ]:
from ortools.linear_solver import pywraplp
location_assignment =[]

def main():
    # Data from all the colours on all the origins
    costs = cost_matrix_reshaped
    num_workers = 71

    num_tasks = len(costs[0])
    #print(num_tasks)
    # Solver
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')


    # Variables
    # x[i, j] is an array of 0-1 variables, which will be 1
    # if worker i is assigned to task j.
    x = {}
    for i in range(num_workers):
        for j in range(num_tasks):
            x[i, j] = solver.IntVar(0, 1, '')

    # Constraints
    # Each worker is assigned to at most 1 task.
    for i in range(num_workers):
        solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

    # Each task is assigned to exactly one worker.
    for j in range(num_tasks):
        solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)

    # Objective
    objective_terms = []
    for i in range(num_workers):
        for j in range(num_tasks):
            objective_terms.append(costs[i][j] * x[i, j])
    solver.Maximize(solver.Sum(objective_terms))

    # Solve
    status = solver.Solve()

    # Print solution.
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print('Total cost = ', solver.Objective().Value(), '\n')
        for i in range(num_workers):
            for j in range(num_tasks):
                # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
                if x[i, j].solution_value() > 0.5:
                    print('Agent %d assigned to Location %d.  Cost = %d' %
                          (i, j, costs[i][j]))
                    location_assignment.append(j)


if __name__ == '__main__':
    main()
    


In [ ]:
Agent_origins = []
for index in location_assignment:
    Agent_origins.append(Selected_positions[index])

In [ ]:
def agent_based_on_origin(Origin,index):
    return agent(np.unravel_index(Origin,init_avail_lattice.shape),stencil_cuboid,index)

In [ ]:
occ_lattice_sim = tg.to_lattice(np.copy(avail_lattice), avail_lattice)
env_P = {"availibility": avail_lattice,"enviornment": Private_sector_Owned}
env_S = {"availibility": avail_lattice,"enviornment": Social_Sector_Rental}
env_F = {"availibility": avail_lattice,"enviornment": Free_Sector_rental}
env_R = {"availibility": avail_lattice,"enviornment": Restaurants_and_Cafes}
env_O = {"availibility": avail_lattice,"enviornment": Offices}
env_RE = {"availibility": avail_lattice,"enviornment": Retail}
env_PR = {"availibility": avail_lattice,"enviornment": Parking}

In [ ]:
Origins_P = Agent_origins[:21]
print(Origins_P)
Agents_P =[]
env_1 =[]
for i,item in enumerate(Origins_P):
    created_agent_1= agent_based_on_origin(item,10)
    Agents_P.append(created_agent_1)
    env_1.append(environment(env_P,created_agent_1,20,"one_neighbour_occupy_cubish_behaviour"))

In [ ]:
Origins_S = Agent_origins[21:37]
print(Origins_S)
Agents_S =[]
env_2 =[]
for item in Origins_S:
    created_agent_2 = agent_based_on_origin(item,100)
    Agents_S.append(created_agent_2)
    env_2.append(environment(env_S,created_agent_2,20,"one_neighbour_occupy_cubish_behaviour"))

In [ ]:
Origins_F = Agent_origins[37:58]
print(Origins_F)
Agents_F =[]
env_3 =[]
for item in Origins_F:
    created_agent_3 = agent_based_on_origin(item,200)
    Agents_F.append(created_agent_3)
    env_3.append(environment(env_F,created_agent_3,20,"one_neighbour_occupy_cubish_behaviour"))

In [ ]:
Origins_R = Agent_origins[59]
print(Origins_R)
Agents_R =agent_based_on_origin(item,300)
env_4= environment(env_R,Agents_R,20,"one_neighbour_occupy_cubish_behaviour")

In [ ]:
Origins_O = Agent_origins[59:63]
print(Origins_O)
Agents_O =[]
env_5 =[]
for item in Origins_O:
    created_agent_5 = agent_based_on_origin(item,400)
    Agents_O.append(created_agent_5)
    env_5.append(environment(env_O,created_agent_5,20,"one_neighbour_occupy_cubish_behaviour"))

In [ ]:
Origins_RE = Agent_origins[63:68]
print(Origins_RE)
Agents_RE =[]
env_6 =[]
for item in Origins_RE:
    created_agent_6 = agent_based_on_origin(item,500)
    Agents_RE.append(created_agent_6)
    env_6.append(environment(env_RE,created_agent_6,20,"one_neighbour_occupy_cubish_behaviour"))

In [ ]:
Origins_PR = Agent_origins[68:71]
print(Origins_PR)
Agents_PR =[]
env_7 =[]
for item in Origins_PR:
    created_agent_7 = agent_based_on_origin(item,600)
    Agents_PR.append(created_agent_7)
    env_7.append(environment(env_PR,created_agent_7,20,"one_neighbour_occupy_cubish_behaviour"))    

In [ ]:
all_enviornments= np.concatenate(((np.array(env_1)),(np.array(env_2)),(np.array(env_3)),(np.array(env_5)),(np.array(env_6)),(np.array(env_7))))
all_enviornments_2 = np.append(all_enviornments,np.array(env_4))

In [ ]:
len(all_enviornments_2)

In [ ]:
env_availability_viz = []
env_list = all_enviornments_2
number_steps = max(map(lambda e:e.number_of_iterations,env_list))
for a in range(number_steps):
    # print(env.availibility)
    #print(env.agent_origin)
    print(a)
    for e in env_list:
        if a < e.number_of_iterations :
            #print(a)
            #print(e.number_of_iterations)
            if e.method_name == "one_neighbour_occupy_squareness_moore":
                e.one_neighbour_occupy_squareness_moore()
                
            elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                e.one_neighbour_occupy_cubish_agents()
                
            elif e.method_name == "random_occupy_squareness_agents" :
                e.random_occupy_squareness_agents()
            
            elif e.method_name == "random_occupy_cubish_agents" :
                e.random_occupy_cubish_agents()  
                
            elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                e.random_occupy_cubish_von_neumann_agents()                           
                
            elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                e.one_neighbour_occupy_squareness_von_neumann()                
                
                
            elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                e.one_neighbour_occupy_squareness_behaviour()  
                
            elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                e.one_neighbour_occupy_cubish_behaviour()  
            
            elif e.method_name == "argmax_occupy_von_neumann" :
                e.argmax_occupy_von_neumann()  
                
            env_availability_viz.append(e.availibility-1)

In [ ]:
p = pv.Plotter(notebook=True)

base_lattice = env_availability_viz[0]
print(base_lattice.unit)
# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

def create_mesh(value):
    f = int(value)
    lattice = env_availability_viz[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([1.1, avail_lattice.size])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return
number_steps_2 = 1500
p.add_slider_widget(create_mesh, [0, number_steps_2], title='Time', value=0, event_type="always", style="classic")


p.show(use_ipyvtk=True)


In [ ]:
"""
for i, lattice in enumerate(env_availability_viz):
    csv_path = os.path.relpath('csv/abm_f_'+ f'{i:03}' + '.csv')
    lattice.to_csv(csv_path)
"""